In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import re
import sys
import os, os.path
import shutil

from collections import Counter
import pickle
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
MIN_TOKEN_FREQ = 100
MAX_TOKENS_IN_FILE = 10000
MIN_EXAMPLES_PER_CODE = 1000
TEST_SIZE = 0.2

In [4]:
CPT_FILE_PATH = "drive/MyDrive/mimic-iii/CPTEVENTS.csv"
DIAGNOSIS_FILE_PATH = "drive/MyDrive/mimic-iii/DIAGNOSES_ICD.csv"
PROCEDURES_FILE_PATH = "drive/MyDrive/mimic-iii/PROCEDURES_ICD.csv"
CORPUS_FILE_PATH = "drive/MyDrive/mimic-iii/cuis" 

# Load the MIMIC-III data and pretrain word embeddings
This first cell in this section is the same as the ICDLoader in NNModels.ipynb. In colab env, it's challenging to import classes from other py files - apologies for the duplicate code. We use this class in this notebook to pretrain a word2vec and a doc2vec model, which will be used later in NNModels.ipynb and SVM_Models.ipynb.

In [5]:
class ICDLoader: 
  """Load ICD billing codes labels for each patient"""

  def __init__(self, corpus_file_path, cpt_file_path, diagnosis_file_path, procedures_file_path, min_examples_per_code, min_token_freq, max_tokens_in_file):
    self.corpus_path = corpus_file_path
    self.cpt_path = cpt_file_path
    self.diagnosis_path = diagnosis_file_path
    self.procedures_path = procedures_file_path
   
    self.max_tokens_in_file = max_tokens_in_file
    self.min_examples_per_code = min_examples_per_code
    self.min_token_freq = min_token_freq
    self.patient2label_dict = None #mapping from patient -> ICD9 label codes
    self.label2idx_dict = None #mapping from ICD9 label code -> embedding idx

    self.token2int = {}

  def df_make_code(self, df, icd_type, code_len, code_col):
    #making special short string for codes
    df['short_code'] = icd_type + "_" + df[code_col].astype(str).str[:code_len]
    return df
  
  def get_label2freq_df(self, df, min_examples_per_code):
    #get a filtered label codes -> frequency mapping table 
    label2freq_df = df[["SUBJECT_ID", "short_code"]].groupby("short_code").nunique()
    label2freq_df = label2freq_df[label2freq_df["SUBJECT_ID"]>min_examples_per_code]
    label2freq_df.rename({"SUBJECT_ID": "freq"}, axis=1, inplace=True)
    return label2freq_df.reset_index()

  def get_patient2label_df(self, df, label2freq_df):
    #get df of patient mapping to all their filtered ICD9 code labels (filtered by freq)
    df_filtered = df[df["short_code"].isin(label2freq_df["short_code"])] 
    patient2label_df = df_filtered[["SUBJECT_ID", "short_code"]]\
                          .groupby("SUBJECT_ID")\
                          .agg({'short_code':lambda sf: set(sf)})
    patient2label_df.rename({"short_code":"short_codes"}, axis=1, inplace=True)          
    return patient2label_df.reset_index()

  def create_patient_label_vec(self, subj_id, patient2label_dict, label2idx_dict):
    #make patient label vector
    code_vec = [0]*len(label2idx_dict)
    codes = patient2label_dict[subj_id]
    for code in codes:
      code_vec[label2idx_dict[code]] = 1
    return code_vec

  def make_cui_token2int_mapping(self):
    #count tokens
    i=0
    token_count_dict = {}
    for file in os.listdir(self.corpus_path):
      text = open(os.path.join(self.corpus_path,file)).read()
      tokens = [token for token in text.split()] #assume all cui in file splitted by space
      if len(tokens) > self.max_tokens_in_file:
        continue
      else:
        for token in tokens:
          if token in token_count_dict:
            token_count_dict[token] += 1
          else:
            token_count_dict[token] = 1
    
    #make token2int mapping
    oov_idx = 0
    idx = 1
    self.token2int['oov_word'] = 0
    for token, count in token_count_dict.items():
      if count > self.min_token_freq:
        self.token2int[token] = idx
        idx += 1
  
  def create_cui_input_sequence(self, tokens):
    #create cui_input_sequence from cui tokens
    input = []
    tokens_set = set(tokens)

    for token in tokens_set:
      if token in self.token2int:
        input.append(self.token2int[token])
      else:
        input.append(self.token2int['oov_word'])

    return input

  def run(self):
    #run everything

    #codes init
    cpt = pd.read_csv(self.cpt_path)
    diagnosis = pd.read_csv(self.diagnosis_path)
    procedures = pd.read_csv(self.procedures_path)

    #codes init
    cpt = self.df_make_code(cpt, 'cpt', 5, 'CPT_NUMBER')
    diagnosis = self.df_make_code(diagnosis, 'diag', 3, 'ICD9_CODE')
    procedures = self.df_make_code(procedures, 'proc', 2, 'ICD9_CODE')

    #codes init
    all_codes = cpt[["SUBJECT_ID", "short_code"]]\
              .append(diagnosis[["SUBJECT_ID", "short_code"]], ignore_index=True)\
              .append(procedures[["SUBJECT_ID", "short_code"]], ignore_index=True)

    #codes init
    label2freq_df = self.get_label2freq_df(all_codes, self.min_examples_per_code)
    patient2label_df = self.get_patient2label_df(all_codes, label2freq_df)
    
    #codes init
    self.label2idx_dict = dict(label2freq_df.reset_index()[["short_code", "index"]].values)
    self.patient2label_dict = dict(patient2label_df.values)

    #cui init
    self.make_cui_token2int_mapping()

    codes = []
    cui_inputs = []

    #processing
    for file in os.listdir(self.corpus_path): #list files to run
      text = open(os.path.join(self.corpus_path, file)).read()
      tokens = [token for token in text.split()]
      if len(tokens) > self.max_tokens_in_file: #cui filter
        continue

      subj_id = int(re.findall('\d+(?=.txt)', file)[0])
      if subj_id not in self.patient2label_dict: #icd9 filter
        continue

      #icd9 process  
      code_vec = self.create_patient_label_vec(subj_id, self.patient2label_dict, 
                                               self.label2idx_dict)
      if sum(code_vec) == 0:
        continue
      codes.append(code_vec)

      #cui process
      cui_input = self.create_cui_input_sequence(tokens)
      cui_inputs.append(cui_input)


    return cui_inputs, codes

In [ ]:
loader = ICDLoader(CORPUS_FILE_PATH, CPT_FILE_PATH, DIAGNOSIS_FILE_PATH, PROCEDURES_FILE_PATH, MIN_EXAMPLES_PER_CODE, MIN_TOKEN_FREQ, MAX_TOKENS_IN_FILE)
cui_inputs, codes = loader.run()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [ ]:
# saving dictionary to avoid processing 46k+ files everytime
save_dict_file = open("drive/MyDrive/mimic-iii/token2int_dict.pkl", "wb")
pickle.dump(loader.token2int, save_dict_file)
save_dict_file.close()

with open("drive/MyDrive/mimic-iii/token2int_dict.pkl", "rb") as fp:
  token2int_dict = pickle.load(fp)

In [ ]:
# with open("drive/MyDrive/mimic-iii/patient2label_dict.pkl", "wb") as fp:   #Pickling
#   pickle.dump(loader.patient2label_dict, fp)

# with open("drive/MyDrive/mimic-iii/label2idx_dict.pkl", "wb") as fp:   #Pickling
#   pickle.dump(loader.label2idx_dict, fp)

# with open("drive/MyDrive/mimic-iii/cui_inputs.pkl", "wb") as fp:   #Pickling
#   pickle.dump(cui_inputs, fp)

# with open("drive/MyDrive/mimic-iii/icd9codes.pkl", "wb") as fp:   #Pickling
#   pickle.dump(codes, fp)

with open("drive/MyDrive/mimic-iii/cui_inputs.pkl", "rb") as fp:   #Pickling
  cui_inputs = pickle.load(fp)

with open("drive/MyDrive/mimic-iii/icd9codes.pkl", "rb") as fp:   #Pickling
  codes = pickle.load(fp)

In [ ]:
len(cui_inputs), len(codes)

(41617, 41617)

## Word2Vec Model

In [ ]:
cui_inputs_str = [[str(x) for x in cui_input] for cui_input in cui_inputs]

In [ ]:
len(cui_inputs_str)

41617

In [ ]:
model = Word2Vec(sentences=cui_inputs_str, window=5, size=300, workers=4, min_count=1)

In [ ]:
model.wv.vectors

array([[-1.5117495e-01, -1.5825114e-01, -2.8098634e-01, ...,
        -4.3912478e-02,  2.5941911e-01,  6.2685108e-01],
       [-7.8855312e-01, -2.0701051e+00,  6.7504030e-01, ...,
        -4.8524055e+00, -1.4103217e+00,  2.7832153e+00],
       [ 4.0319734e+00, -1.8833885e+00, -1.8876145e+00, ...,
         5.8676332e-01,  5.2643979e-01,  5.5151582e+00],
       ...,
       [ 6.0104658e-03,  7.8745018e-04, -4.4967472e-03, ...,
        -6.9574029e-03, -1.3573231e-02,  1.0548139e-02],
       [ 1.5774533e-02,  1.0217195e-02, -9.9627888e-03, ...,
        -1.8214449e-02, -3.1583119e-02,  7.8437198e-03],
       [-6.6564404e-03, -1.4135306e-02,  1.6191775e-02, ...,
         2.9815650e-02,  2.6409375e-02, -5.4407003e-03]], dtype=float32)

In [ ]:
model.save("drive/MyDrive/mimic-iii/word2vec_2.model")
# model = Word2Vec.load("drive/MyDrive/mimic-iii/word2vec.model")

## Doc2Vec model

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(cui_inputs_str)]
model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, workers=4)

In [ ]:
model.save("drive/MyDrive/MIMIC/mimic-iii/doc2vec.model")